## In NoteBook

In [1]:
%load_ext zipline

到目前为止，我们已经在研究中创建并运行了一条管道。现在我们转到IDE。首先，我们创建一个骨架算法，导入`Pipeline`，并为其添加一个`make_pipeline`函数，以创建一个空管道。

In [2]:
from zipline.pipeline import Pipeline

def initialize(context):
    my_pipe = make_pipeline()

def make_pipeline():
    return Pipeline()

## Attaching a Pipeline

回想一下，在研究中，我们运行了`make_pipeline`来创建管道对象的实例，并使用`run_pipeline`在指定的日期范围内运行管道。我们不能在算法中安全地做到这一点，我们必须以某种方式允许模拟为我们运行我们的管道。为了让仿真运行我们的管道，我们必须将管道与attach_pipeline连接起来。

`attach_pipeline`函数需要两个参数：对我们的管道对象的引用，以及可以任意决定的管道的字符串名称。让我们导入`attach_pipeline`并将我们的空管道连接到我们的骨架示例中。

In [3]:
from zipline.pipeline import Pipeline
from zipline.api import attach_pipeline

def initialize(context):
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    return Pipeline()

现在管道已连接，它将模拟每一天运行一次。如果算法在2016年6月6日星期一至2016年6月10日星期五进行回测或实时交易，我们的管道将在每周的每一天运行一次（总共5次）。附加管道将每天产生一个新的数据框输出。研究中的日输出与`run_pipeline`的输出类似。但是，我们算法中的输出数据框不包括索引中的日期，因为当前模拟日期暗示为我们管道计算的日期。

## Pipeline Output

管道的输出可以在`before_trading_start`中使用`pipeline_output`进行检索。`pipeline_output`需要附加管道的名称作为参数，并返回模拟中当前日期的输出数据框。让我们导入`pipeline_output`并修改我们的骨架示例，以便每天在上下文中存储我们的管道输出。

In [4]:
#%%zipline --start 2016-1-1 --end 2018-1-1
from zipline.pipeline import Pipeline
from zipline.api import attach_pipeline, pipeline_output

def initialize(context):
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    return Pipeline()

def before_trading_start(context, data):
    # Store our pipeline output DataFrame in context.
    #output = pipeline_output('my_pipeline')
    context.output = pipeline_output('my_pipeline')

我们的骨架示例现在每天生成3000行(期间有效证券数量)和默认指标数量列的empy数据框。 输出数据框将如下所示（请注意，该索引不再像研究中的MultiIndex一样）：

## Using Our Pipeline From Research

为了在我们的算法中包含我们在前一课中创建的管道，我们可以简单地将我们在研究中编写的`make_pipeline`函数以及所需的import语句复制到我们的算法中。以下内容将运行我们的管道，并在每天的上下文中存储150行和2列（多和空）的输出数据框。

In [5]:
%%zipline --start 2016-1-1 --end 2018-1-1
from zipline.pipeline import Pipeline
from zipline.api import attach_pipeline, pipeline_output
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage
from zipline.pipeline.builtin import IsST

def initialize(context):
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    """
    Create our pipeline.
    """

    # Base universe set to the QTradableStocksUS.
    base_universe = IsST()

    # 10-day close price average.
    mean_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=10,
        mask=base_universe
    )

    # 30-day close price average.
    mean_30 = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=30,
        mask=base_universe
    )

    percent_difference = (mean_10 - mean_30) / mean_30

    # Filter to select securities to short.
    shorts = percent_difference.top(75)

    # Filter to select securities to long.
    longs = percent_difference.bottom(75)

    # Filter for all securities that we want to trade.
    securities_to_trade = (shorts | longs)

    return Pipeline(
        columns={
            'mean_10': mean_10,
            #'shorts': shorts
        },
        #screen=(securities_to_trade),
    )

def before_trading_start(context, data):
    # Store our pipeline output DataFrame in context
    output = pipeline_output('my_pipeline')

[2018-04-27 19:47:24.846116] INFO: zipline.finance.metrics.tracker: Simulated 488 trading days
first open: 2016-01-04 01:31:00+00:00
last close: 2017-12-29 07:00:00+00:00


,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2016-01-04 05:35:00+00:00,NaN,0.0,NaN,-0.070206,NaN,NaN,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,1,[],0.0
2016-01-05 07:00:00+00:00,0.0,0.0,0.0,-0.067603,0.819490,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,2,[],0.0
2016-01-06 07:00:00+00:00,0.0,0.0,0.0,-0.051246,0.745908,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,3,[],0.0
2016-01-07 02:00:00+00:00,0.0,0.0,0.0,-0.117026,0.738897,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,4,[],0.0
2016-01-08 07:00:00+00:00,0.0,0.0,0.0,-0.099020,0.732421,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,5,[],0.0
2016-01-11 07:00:00+00:00,0.0,0.0,0.0,-0.144346,0.684357,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,6,[],0.0
2016-01-12 07:00:00+00:00,0.0,0.0,0.0,-0.138112,0.653811,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,7,[],0.0
2016-01-13 07:00:00+00:00,0.0,0.0,0.0,-0.154148,0.605383,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,8,[],0.0
2016-01-14 07:00:00+00:00,0.0,0.0,0.0,-0.136541,0.606174,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,9,[],0.0
2016-01-15 07:00:00+00:00,0.0,0.0,0.0,-0.164105,0.577413,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,10,[],0.0


In [6]:
_

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2016-01-04 05:35:00+00:00,NaN,0.0,NaN,-0.070206,NaN,NaN,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,1,[],0.0
2016-01-05 07:00:00+00:00,0.0,0.0,0.0,-0.067603,0.819490,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,2,[],0.0
2016-01-06 07:00:00+00:00,0.0,0.0,0.0,-0.051246,0.745908,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,3,[],0.0
2016-01-07 02:00:00+00:00,0.0,0.0,0.0,-0.117026,0.738897,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,4,[],0.0
2016-01-08 07:00:00+00:00,0.0,0.0,0.0,-0.099020,0.732421,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,5,[],0.0
2016-01-11 07:00:00+00:00,0.0,0.0,0.0,-0.144346,0.684357,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,6,[],0.0
2016-01-12 07:00:00+00:00,0.0,0.0,0.0,-0.138112,0.653811,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,7,[],0.0
2016-01-13 07:00:00+00:00,0.0,0.0,0.0,-0.154148,0.605383,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,8,[],0.0
2016-01-14 07:00:00+00:00,0.0,0.0,0.0,-0.136541,0.606174,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,9,[],0.0
2016-01-15 07:00:00+00:00,0.0,0.0,0.0,-0.164105,0.577413,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,10,[],0.0


In [7]:
from zipline.research import run_pipeline

In [8]:
from zipline.pipeline import Pipeline
from zipline.api import attach_pipeline, pipeline_output
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage
from zipline.pipeline.builtin import IsST, QTradableStocksUS

def make_pipeline():
    """
    Create our pipeline.
    """

    # Base universe set to the QTradableStocksUS.
    base_universe = ~QTradableStocksUS()

    # 10-day close price average.
    mean_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=10,
        mask=base_universe
    )

    # 30-day close price average.
    mean_30 = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=30,
        mask=base_universe
    )

    percent_difference = (mean_10 - mean_30) / mean_30

    # Filter to select securities to short.
    shorts = percent_difference.top(75)

    # Filter to select securities to long.
    longs = percent_difference.bottom(75)

    # Filter for all securities that we want to trade.
    securities_to_trade = (shorts | longs)

    return Pipeline(
        columns={
            'mean_10': mean_10,
            #'shorts': shorts
        },
        screen=base_universe,
    )

In [9]:
result = run_pipeline(make_pipeline(), '2018-04-23', '2018-04-26')
result

TypeError: unsupported operand type(s) for &: 'int' and 'V'

In [ ]:
from zipline.research.utils import select_output_by

In [ ]:
select_output_by(result,'2018-04-23','2018-04-24',stock_codes=['000585','600871'])

In [ ]:
stock_codes = ['601106','000526']